In [1]:
import requests
import pandas as pd
import json
from openai import OpenAI
from process import get_tasks, add_task, update_task, delete_task

client = OpenAI()

In [2]:
oauthz_demo_get_desc = """
The AI agent triggers the OAuth2.0 authentication endpoint when users request login or access to protected resources. 
It listens for commands like "log me in," "authenticate me," or "sign in to my account." 
Upon receiving the request, the agent redirects the user to an authorization page where they log in and grant access. 
If successful, the agent extracts the authorization code from the redirected URL for further processing. 
The endpoint requires parameters like response_type=code, client_id, redirect_uri, and scope=openid profile.
"""  

oauthz_demo_post_desc = """
The AI agent triggers the OAuth2.0 token exchange endpoint after receiving the authorization code.
It listens for commands like "Complete my login" or "Authenticate my session." Once the authorization code is available, the agent sends a POST 
request to exchange it for an access token and a refresh token. These tokens allow access to user APIs and are used for subsequent requests.
Required parameters include grant_type=authorization_code, redirect_uri, and code, with Basic Authentication using client_id and client_secret.
"""  

oauthz_demo_pkce_get_desc = """
The AI agent triggers the PKCE-based authentication endpoint when users request secure login without a client secret. 
It listens for phrases like "Log in securely," "Authenticate without client secret," or "Authorize with PKCE." 
The agent redirects the user to the authentication page, where they log in and grant permissions. On success, an authorization code is returned
to the redirect URI. The agent then extracts the code, which can be exchanged for an access token. Required parameters include response_type=code, 
client_id, redirect_uri, scope=openid profile email, and code_challenge.
"""  

oauthz_demo_pkce_post_desc = """
The AI agent triggers the PKCE token exchange endpoint after receiving the authorization code. It listens for commands like "Complete my login with 
PKCE," "Exchange my PKCE auth code for a token," or "Get an access token using PKCE." The agent sends a POST request with the authorization code and 
verifies the code_verifier against the code_challenge. If successful, the agent receives an access token and refresh token. Required parameters include 
grant_type=authorization_code, redirect_uri, code, and code_verifier, with Basic Authentication using client_id and client_secret.
"""  


In [3]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "oauthz_demo_get",
            "description": oauthz_demo_get_desc,
            "parameters": {"type": "object", "properties": {}, "required": [], "additionalProperties": False},
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "oauthz_demo_post",
            "description": oauthz_demo_post_desc,
            "parameters": {"type": "object", "properties": {}, "required": [], "additionalProperties": False},
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "oauthz_demo_pkce_get",
            "description": oauthz_demo_pkce_get_desc,
            "parameters": {"type": "object", "properties": {}, "required": [], "additionalProperties": False},
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "oauthz_demo_pkce_post",
            "description": oauthz_demo_pkce_post_desc,
            "parameters": {"type": "object", "properties": {}, "required": [], "additionalProperties": False},
            "strict": True,
        },
    },
]

In [4]:
def oauthz_demo_get():
    print('oauthz_demo_get is called')

def oauthz_demo_post():
    print('oauthz_demo_post is called')

def oauthz_demo_pkce_get():
    print('oauthz_demo_pkce_get is called')

def oauthz_demo_pkce_post():
    print('oauthz_demo_pkce_post is called')


In [5]:
function_map = {
    'oauthz_demo_get':oauthz_demo_get,
    'oauthz_demo_post':oauthz_demo_post,
    'oauthz_demo_pkce_get':oauthz_demo_pkce_get,
    'oauthz_demo_pkce_post':oauthz_demo_pkce_post
}

In [6]:
def ask_gpt(msg):
    messages = [{"role": "user", "content": msg}]

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
    )

    tool_call = completion.choices[0].message.tool_calls
    return tool_call


In [9]:
query = 'auth login user'
tool_calls = ask_gpt(query)

for call in tool_calls:
    function_map[call.function.name]()

oauthz_demo_get is called


In [1]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

def create_pingfederate_admin_agent():
    """
    Returns a LangChain agent that can call the PingFederate admin API tools.
    """
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-3.5-turbo",  # Specify the model
    )
    
    # Instantiate all tools as Tool objects (Ensure tools take a single dictionary input)
    tools = [
        Tool(
            name="ListAdminAccounts",
            func=ListAdminAccountsTool().run,
            description="Lists all admin accounts."
        ),
        Tool(
            name="CreateAdminAccount",
            func=CreateAdminAccountTool().run,
            description="Creates a new admin account. Provide required details as input."
        ),
        Tool(
            name="GetAdminAccount",
            func=GetAdminAccountTool().run,
            description="Retrieves details of a specific admin account by ID."
        ),
        Tool(
            name="UpdateAdminAccount",
            func=UpdateAdminAccountTool().run,
            description="Updates an existing admin account. Provide the necessary fields."
        ),
        Tool(
            name="DeleteAdminAccount",
            func=DeleteAdminAccountTool().run,
            description="Deletes an admin account using its ID."
        ),
        Tool(
            name="ResetPassword",
            func=ResetPasswordTool().run,
            description="Resets the password for an admin account."
        ),
        Tool(
            name="ChangeMyPassword",
            func=ChangeMyPasswordTool().run,
            description="Allows an admin to change their own password."
        )
    ]

    # Use OpenAI functions agent (supports multi-input tools)
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.OPENAI_FUNCTIONS,  # Supports structured input tools
        verbose=True
    )
    
    return agent
